In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# Macronutrients & Ratings: 
# Impact of High-Carbs and Low-Protein on Recipe Ratings

**Name(s)**: Ananya Krishnan, John Wesley Pabalate

**Website Link**: https://j0hnwesl3yyy.github.io/recipe-rating-analysis/

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

import plotly.express as px
import plotly.graph_objects as go

pd.options.plotting.backend = 'plotly'

#from dsc80_utils import * # Feel free to uncomment and use this.

## Step 1: Introduction

## Question: 
- Do high-carb, low-protein recipes receive significantly different ratings compared to other recipes? 

## Step 2: Data Cleaning and Exploratory Data Analysis

In [ ]:
#JohnWesley's Directory
#recipes = pd.read_csv('/Users/johnwesleypabalate/Desktop/dsc80-2025-wi/projects/project04/data/RAW_recipes.csv')

#Ananya's Directory
recipes = pd.read_csv('/Users/ananyakrishnan/Downloads/DSC80/projects/project04/data/RAW_recipes.csv')
recipes.head()

#Colab Directory
# recipes = pd.read_csv('/content/RAW_recipes.csv')
recipes.head()

In [ ]:
#JohnWesley's Directory
#reviews = pd.read_csv('/Users/johnwesleypabalate/Desktop/dsc80-2025-wi/projects/project04/data/RAW_interactions.csv')

#Ananya's Directory
reviews = pd.read_csv('../DSC80/projects/project04/data/RAW_interactions.csv')
reviews.head()

#Colab Directory
# reviews = pd.read_csv('/content/interactions.csv')
reviews.head()

Let us now merge recipes and ratings into one comprehensive dataset.

In [ ]:
recipe_ratings = recipes.merge(reviews, left_on = 'id', right_on = 'recipe_id', how="left")
recipe_ratings.head()

Let us replace all 0s in the ratings column with NaN values. The 0 represents no rating given, but it will influence any calculations we perform with the ratings. We also calculate the average ratings for each recipe and store it in `avg_recipe_rating`. We will then add it as a column to recipe_reviews.

In [ ]:
recipe_ratings.loc[recipe_ratings['rating'] == 0, 'rating'] = np.nan
avg_recipe_rating = recipe_ratings.groupby('recipe_id')['rating'].mean()

In [ ]:
recipe_ratings = recipe_ratings.merge(avg_recipe_rating.reset_index().rename(columns={'rating': 'avg_rating'}), on = 'recipe_id')

In [ ]:
#Made a copy of recipe_ratings for the purpose of using it for the Missingness and Baseline: 
merged_df = recipe_ratings.copy()
merged_df = merged_df.drop(columns=['Unnamed: 0'])
merged_df.head()

There are many columns not relevant to our question, so we will clean only the columns related to recipe id, nutrition information and ratings.

In [ ]:
original_df = merged_df.copy()

In [ ]:
recipe_ratings = recipe_ratings.drop(columns = ['recipe_id']).rename(columns = {'id': 'recipe_id'}).drop(columns = ['Unnamed: 0'])

In [ ]:
original_df = original_df.drop(columns = ['recipe_id']).rename(columns = {'id': 'recipe_id'})

Let us now look at the columns we have and clean them up one by one.

In [ ]:
recipe_ratings.dtypes

We observe that `nutrition` actually contains strings formatted to look like lists, so let us convert it to real lists.

In [ ]:
recipe_ratings['nutrition'] = recipe_ratings['nutrition'].str.strip('[').str.strip(']').str.replace("'", "").str.split(', ')

The `nutrition` column now contains lists of values. Let us separate each value into its respective category - `'calories'`, `'total_fat'`, `'sugar'`, `'sodium'`, `'protein'`, `'saturated_fat'` and `'carbohydrates'`. We can then drop the `nutrition` column.

In [ ]:
categories = ['calories', 'total_fat', 'sugar', 'sodium', 'protein', 'saturated_fat', 'carbohydrates']
recipe_ratings = recipe_ratings.assign(
    **{category: pd.to_numeric(recipe_ratings['nutrition'].str[i], errors='coerce') for i, category in enumerate(categories)}
)
recipe_ratings = recipe_ratings.drop(columns = ['nutrition'])

Let us now look at our cleaned dataset:

In [ ]:
original_df = pd.concat([original_df, recipe_ratings[['calories', 'total_fat', 'sugar', 'sodium', 'protein', 'saturated_fat', 'carbohydrates']]], axis=1).drop(columns = ['nutrition'])

In [ ]:
original_df.shape

In [ ]:
recipe_ratings.isna().sum(axis = 0)

In [ ]:
recipe_ratings.describe()

The nutritional values seem to have abnormally high max values despite a reasonable mean. Let us look at the rows with high protein or high carbohydrate values

In [ ]:
recipe_ratings[(recipe_ratings['protein'] > 200) | (recipe_ratings['carbohydrates'] > 200)]

These rows have proportionally high calories, meaning this is unlikely to be an error and could just be because of large portion sizes. We can leave it as it is.

We want to account for the percentge of calories that the protein and carbohydrate contribute to. We will define high carbohydrate - low protein recipes using arbitrary cutoffs, considering those in the top 25th percentile of carb percentage and bottom 25th percentile of protein percentage. Using percentiles ensures that we select recipes that are high-carb in absolute terms and low-protein in absolute terms. This guarantees that the selected recipes are truly high in carbohydrate content and low in protein content, rather than just having a high ratio.

To do this, we first need to convert protein and carbohydrate to calories. Each gram of carbohydrate or protein contains 4 calories.

In [ ]:
carb_calories = recipe_ratings['carbohydrates'] * 4  # 4 calories per gram of carbs
protein_calories = recipe_ratings['protein'] * 4  # 4 calories per gram of protein

recipe_ratings['carb_prop'] = carb_calories / recipe_ratings['calories']
recipe_ratings['protein_prop'] = protein_calories / recipe_ratings['calories']

In [ ]:
carb_threshold = recipe_ratings['carb_prop'].quantile(0.75)
protein_threshold = recipe_ratings['protein_prop'].quantile(0.25)

recipe_ratings['high_carb_low_protein'] = (
    (recipe_ratings['carb_prop'] >= carb_threshold) &
    (recipe_ratings['protein_prop'] <= protein_threshold)
)

In [ ]:
recipe_ratings.describe()

In [ ]:
original_df = pd.concat([original_df, recipe_ratings[['carb_prop', 'protein_prop', 'high_carb_low_protein']]], axis=1)

We can see that protein_percent has a max value of 1.88 which is not possible, indicating an error. We will drop all such rows.

In [ ]:
recipe_ratings = recipe_ratings[(recipe_ratings['protein_prop'] <= 1)]

In [ ]:
original_df = original_df[(original_df['protein_prop'] <= 1)]

Now we are ready to visualize our features.

### Univariate Analysis

In [ ]:
# import plotly.io as pio
# pio.renderers.default = "browser"  # IGNORE THIS FOR NOW

In [ ]:
#Distribution of Ratings
fig1 = px.histogram(recipe_ratings, x='avg_rating', nbins=10, title='Distribution of Average Recipe Ratings')
# fig1.write_html("assets/avg_rating_histogram.html", include_plotlyjs="cdn")

# Create and save the second histogram
fig2 = px.histogram(recipe_ratings, x='protein_prop', nbins=10, title='Distribution of Protein Proportion in Recipes')
fig2.write_html("/Users/johnwesleypabalate/Desktop/recipe-rating-analysis/assets/protein_prop_histogram.html", include_plotlyjs="cdn")

The distribution of average ratings is **highly skewed to the left**.This suggests that most recipes receive **high ratings**, making it important to analyze which rating values appear most frequently and how they relate to other factors like the nutritional facts of the food.

Most ratings left by people tend to be 5 stars.

Now let's look at the distribution of Carbohydrate and Protein content of recipes

In [ ]:
recipe_ratings['protein'].describe()

In [ ]:
recipe_ratings['carbohydrates'].describe()

There appears to be very high values of carb and protein (over 3000) which seems unrealistic.

In [ ]:
fig_protein = px.box(recipe_ratings, x='protein', title='Boxplot of Protein Content')
fig_protein.show()

fig_carbs = px.box(recipe_ratings, x='carbohydrates', title='Boxplot of Carbohydrates Content')
fig_carbs.show()


In [ ]:
recipe_ratings[(recipe_ratings['carbohydrates'] > 200) | (recipe_ratings['carbohydrates'] > 200)].shape[0] / recipe_ratings.shape[0] * 100

Less than 0.3 % of the data has either protein or carbohydrate content over 200g, so we can leave the outliers as they are since they are not likely to affect our analyses

### Bivariate Analysis

In [ ]:
px.scatter(recipe_ratings, x="avg_rating", y="carbohydrates",
                 title="Carbohydrates vs. Ratings").show()


In [ ]:
px.scatter(recipe_ratings, x="avg_rating", y="protein",
                 title="Protein vs. Ratings").show()

In [ ]:
# Corrected scatter plots using proportions
fig1 = px.scatter(recipe_ratings, x="avg_rating", y="carb_prop",
                  title="Carbohydrate Proportion vs. Ratings",
                  opacity=0.5)
fig1.show()

fig2 = px.scatter(recipe_ratings, x="avg_rating", y="protein_prop",
                  title="Protein Proportion vs. Ratings",
                  opacity=0.5)
fig2.show()


In [ ]:
fig = px.density_heatmap(recipe_ratings, x="avg_rating", y="carb_prop",
                         title="Heatmap of Carbohydrate Proportion vs. Ratings",
                         nbinsx=10, nbinsy=30, color_continuous_scale="Blues")  # Increase bins
fig.show()


In [ ]:
fig = px.density_heatmap(recipe_ratings, x="avg_rating", y="protein_prop",
                         title="Heatmap of Carbohydrate Proportion vs. Ratings",
                         nbinsx=10, nbinsy=30, color_continuous_scale="Blues")  # Increase bins
fig.show()

In [ ]:
fig1 = px.box(recipe_ratings, x="avg_rating", y="carb_prop",
              title="Carbohydrate Proportion by Ratings",
              labels={"avg_rating": "Average Rating", "carb_prop": "Carbohydrate Proportion"})
fig1.show()

fig2 = px.box(recipe_ratings, x="avg_rating", y="protein_prop",
              title="Protein Proportion by Ratings",
              labels={"avg_rating": "Average Rating", "protein_prop": "Protein Proportion"})
fig2.show()


## Interesting Aggregates

Semething interesting to see would be if carbohydrate and protein content vary by how long it takes to make a recipe. Do shorter recipes tend to be carb-heavy while longer recipes are more balanced?

In [ ]:
recipe_ratings['cooking_time_category'] = pd.cut(recipe_ratings['minutes'], 
                                                 bins=[0, 30, 60, 120, recipe_ratings['minutes'].max()], 
                                                 labels=['<30 min', '30-60 min', '60-120 min', '120+ min'],
                                                 include_lowest=True)

agg_cooking_time = recipe_ratings.groupby('cooking_time_category').agg(
    avg_rating=('avg_rating', 'mean'),
    avg_carb_prop=('carb_prop', 'mean'),
    avg_protein_prop=('protein_prop', 'mean'),
    count=('recipe_id', 'count')
).reset_index()


agg_cooking_time


In [ ]:
fig = px.bar(agg_cooking_time, x='cooking_time_category', y=['avg_carb_prop', 'avg_protein_prop'],
             title='Average Macronutrient Proportions by Cooking Time Category',
             labels={'value': 'Proportion', 'cooking_time_category': 'Cooking Time Category'},
             barmode='group')
fig.show()


Interestingly, it looks like the carbohydrate content stays around the same through dfferent cooking time categories, but protein content seems to increase. 

## Step 3: Assessment of Missingness

In [ ]:
# merged_df.isnull().sum()

In [ ]:
merged_df.isnull().sum()

In [ ]:
# Define missingness indicator for "review"
merged_df["review_missing"] = merged_df["review"].isnull()

# Number of repetitions
n_repetitions = 500
shuffled = merged_df.copy()

tvds = []
for _ in range(n_repetitions):

    # Shuffling the missingness indicator for 'review'
    shuffled["review_missing"] = np.random.permutation(shuffled["review_missing"])

    # Computing and storing the TVD
    pivoted = (
        shuffled
        .pivot_table(index="rating", columns="review_missing", aggfunc="size")
    )

    pivoted = pivoted / pivoted.sum()

    tvd = pivoted.diff(axis=1).iloc[:, -1].abs().sum() / 2
    tvds.append(tvd)

# Compute observed TVD
observed_pivot = merged_df.pivot_table(index="rating", columns="review_missing", aggfunc="size")
observed_pivot = observed_pivot / observed_pivot.sum()
observed_tvd = observed_pivot.diff(axis=1).iloc[:, -1].abs().sum() / 2

# Create a histogram of TVD values
fig = px.histogram(pd.DataFrame(tvds), x=0, nbins=50, histnorm="probability",
                   title="Empirical Distribution of the TVD")

# Add observed TVD as a red vertical line
fig.add_vline(x=observed_tvd, line_color="red", line_width=2, opacity=1)

# Add annotation for observed TVD
fig.add_annotation(text=f'<span style="color:red">Observed TVD = {round(observed_tvd, 2)}</span>',
                   x=2.5 * observed_tvd, showarrow=False, y=0.16)

# Adjust axis range
fig.update_layout(yaxis_range=[0, 0.2])

# Show the plot
fig.show()

In [ ]:
np.mean(np.array(tvds) >= observed_tvd)

We **fail to reject the null**.
This test does not provide evidence that the missingness in the 'review' column is dependent on 'rating' since 0.69 > 0.05

In [ ]:
merged_df["review_missing"] = merged_df["review"].isnull()

# Number of repetitions
n_repetitions = 500
shuffled = merged_df.copy()

tvds = []
for _ in range(n_repetitions):

    # Shuffling the missingness indicator for 'review'
    shuffled["review_missing"] = np.random.permutation(shuffled["review_missing"])

    # Computing and storing the TVD
    pivoted = (
        shuffled
        .pivot_table(index="n_steps", columns="review_missing", aggfunc="size")
    )

    pivoted = pivoted / pivoted.sum()

    tvd = pivoted.diff(axis=1).iloc[:, -1].abs().sum() / 2
    tvds.append(tvd)

# Compute observed TVD
observed_pivot = merged_df.pivot_table(index="n_steps", columns="review_missing", aggfunc="size")
observed_pivot = observed_pivot / observed_pivot.sum()
observed_tvd = observed_pivot.diff(axis=1).iloc[:, -1].abs().sum() / 2

# Compute p-value
p_value = np.mean(np.array(tvds) >= observed_tvd)

# Create a histogram of TVD values
fig = px.histogram(pd.DataFrame(tvds), x=0, nbins=50, histnorm="probability",
                   title="Empirical Distribution of the TVD for n_steps")

# Add observed TVD as a red vertical line
fig.add_vline(x=observed_tvd, line_color="red", line_width=2, opacity=1)

# Add annotation for observed TVD
fig.add_annotation(text=f'<span style="color:red">Observed TVD = {round(observed_tvd, 2)}</span>',
                   x=2.5 * observed_tvd, showarrow=False, y=0.16)

# Adjust axis range
fig.update_layout(yaxis_range=[0, 0.2])

# Show the plot
fig.show()

In [ ]:
np.mean(np.array(tvds) >= observed_tvd)


In [ ]:
merged_df["rate_missing"] = merged_df["rating"].isnull()

# Number of repetitions
n_repetitions = 500
shuffled = merged_df.copy()

tvds = []
for _ in range(n_repetitions):

    # Shuffling the missingness indicator for 'review'
    shuffled["rate_missing"] = np.random.permutation(shuffled["rate_missing"])

    # Computing and storing the TVD
    pivoted = (
        shuffled
        .pivot_table(index="n_steps", columns="rate_missing", aggfunc="size")
    )

    pivoted = pivoted / pivoted.sum()

    tvd = pivoted.diff(axis=1).iloc[:, -1].abs().sum() / 2
    tvds.append(tvd)

# Compute observed TVD
observed_pivot = merged_df.pivot_table(index="n_steps", columns="rate_missing", aggfunc="size")
observed_pivot = observed_pivot / observed_pivot.sum()
observed_tvd = observed_pivot.diff(axis=1).iloc[:, -1].abs().sum() / 2

# Compute p-value
p_value = np.mean(np.array(tvds) >= observed_tvd)

# Create a histogram of TVD values
fig = px.histogram(pd.DataFrame(tvds), x=0, nbins=50, histnorm="probability",
                   title="Empirical Distribution of the TVD for n_steps")

# Add observed TVD as a red vertical line
fig.add_vline(x=observed_tvd, line_color="red", line_width=2, opacity=1)

# Add annotation for observed TVD
fig.add_annotation(text=f'<span style="color:red">Observed TVD = {round(observed_tvd, 2)}</span>',
                   x=2.5 * observed_tvd, showarrow=False, y=0.16)

# Adjust axis range
fig.update_layout(yaxis_range=[0, 0.2])

# Show the plot
fig.show()

In [ ]:
merged_df["rate_missing"] = merged_df["rating"].isnull()

# Number of repetitions
n_repetitions = 500
shuffled = merged_df.copy()

tvds = []
for _ in range(n_repetitions):

    # Shuffling the missingness indicator for 'review'
    shuffled["rate_missing"] = np.random.permutation(shuffled["rate_missing"])

    # Computing and storing the TVD
    pivoted = (
        shuffled
        .pivot_table(index="description", columns="rate_missing", aggfunc="size")
    )

    pivoted = pivoted / pivoted.sum()

    tvd = pivoted.diff(axis=1).iloc[:, -1].abs().sum() / 2
    tvds.append(tvd)

# Compute observed TVD
observed_pivot = merged_df.pivot_table(index="description", columns="rate_missing", aggfunc="size")
observed_pivot = observed_pivot / observed_pivot.sum()
observed_tvd = observed_pivot.diff(axis=1).iloc[:, -1].abs().sum() / 2

# Compute p-value
p_value = np.mean(np.array(tvds) >= observed_tvd)

# Create a histogram of TVD values
fig = px.histogram(pd.DataFrame(tvds), x=0, nbins=50, histnorm="probability",
                   title="Empirical Distribution of the TVD for n_steps")

# Add observed TVD as a red vertical line
fig.add_vline(x=observed_tvd, line_color="red", line_width=2, opacity=1)

# Add annotation for observed TVD
fig.add_annotation(text=f'<span style="color:red">Observed TVD = {round(observed_tvd, 2)}</span>',
                   x=2.5 * observed_tvd, showarrow=False, y=0.16)

# Adjust axis range
fig.update_layout(yaxis_range=[0, 0.2])

# Show the plot
fig.show()

We **reject the null**.
This test provide evidence that the missingness in the 'review' column is dependent on 'n_steps' since 0.118 < 0.24

## Step 4: Hypothesis Testing

Our goal is to see if carbohydrate and protein content affect ratings of recipes. We define high-carb, low-protein recipes as those that fall into both:

- The top 25th percentile for the proportion of calories from carbohydrates
- The bottom 25th percentile for the proportion of calories from protein

**Null Hypothesis (H₀):** Recipes with high carb % and low protein % receive the same ratings as other recipes.  

**Alternative Hypothesis (Hₐ):** Recipes with high carb % and low protein % receive significantly different ratings.

**Test statistic:** Mean difference in ratings between the high-carb, low-protein group and others.

**Significance level:** 0.05

In [ ]:
original_df.columns

In [ ]:
observed_diff = recipe_ratings.groupby("high_carb_low_protein")["avg_rating"].mean().diff().iloc[-1]

def permute_ratings(df):
    shuffled = df["avg_rating"].sample(frac=1, replace=False).reset_index(drop=True)
    df["shuffled_rating"] = shuffled
    return df.groupby("high_carb_low_protein")["shuffled_rating"].mean().diff().iloc[-1]

perm_diffs = [permute_ratings(recipe_ratings) for _ in range(1000)]

p_value = np.mean(np.array(perm_diffs) >= observed_diff)
print("P-value:", p_value)

Since the p-value is less than the significance level 0.05, we reject the null.

## Step 5: Framing a Prediction Problem

We want to predict the ratings of recipes using different features.

## Step 6: Baseline Model

In [ ]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_absolute_error, r2_score
import plotly.express as px

We will use all data we have engineered along with the original provided data. We have been keeping track of this in `original_df` so we will use this dataframe for our modeling. Let us first create some new features of interest:

`'steps_per_minute'`: We will create this column to put into perspective the number of steps required for the duration of a recipe. This tells us whether it requires higher effort (having to perform several steps quickly). Effort may be related to how a recipe is rated. We will use this for our final analysis.

Let us also convert 'high_carb_low_protein' columnn into a binary column to make our analyses easier. We will then drop all null values so that they dont affect the modeling.

In [ ]:
original_df['steps_per_minute'] = original_df['n_steps'] / (original_df['minutes'] + 1e-6)
original_df['high_carb_low_protein'] = original_df['high_carb_low_protein'].astype(int)
original_df = original_df.dropna()

original_df

For our baseline model, we will use 'minutes' (quantitative, numerical), 'n_ingredients'(quantitative, numerical) and 'high_carb_low_protein'(quantitative, nominal) in a Random Forest Regressor model to predict 'avg_rating'. 

We want to use these features because we believe the recipes that take very long may get lower ratings while higher number of ingredients may make the recipe more complex and taste better, receiving higher ratings. From our previous analyses we saw that high carb, low protein recipes tend to get higher ratings, so we beleive these would be a good predictors.

We standardize the quantitative features using StandardScaler to ensure they are on a comparable scale. The categorical feature is left as-is since it is already binary.

In [ ]:
baseline_features = ['minutes', 'n_ingredients', 'high_carb_low_protein']
final_features = ['high_carb_low_protein', 'steps_per_minute', 'carb_prop', 'protein_prop']

y = original_df['avg_rating'] 
X = original_df.drop(columns=['avg_rating', 'rating']) 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_baseline = X_train[baseline_features]
X_test_baseline = X_test[baseline_features]
X_train_final = X_train[final_features]
X_test_final = X_test[final_features]

preprocessor_baseline = ColumnTransformer([
    ('num', StandardScaler(), ['minutes', 'n_ingredients']) 
], remainder='passthrough')

preprocessor_final = ColumnTransformer([
    ('num', StandardScaler(), ['steps_per_minute', 'carb_prop', 'protein_prop']),  
], remainder='passthrough')


In [ ]:
baseline_pipeline = Pipeline([
    ('preprocessor', preprocessor_baseline),
    ('model', RandomForestRegressor(random_state=42, n_jobs=-1))  
])

baseline_pipeline.fit(X_train_baseline, y_train)

y_pred_baseline = baseline_pipeline.predict(X_test_baseline)
mae_baseline = mean_absolute_error(y_test, y_pred_baseline)
r2_baseline = r2_score(y_test, y_pred_baseline)

print(f"Baseline MAE: {mae_baseline:.4f}")
print(f"Baseline R²: {r2_baseline:.4f}")

## Step 7: Final Model

For the final model, we will use more nutrition related features, since they might be more likely to affect quality of food and hence the ratings:
'high_carb_low_protein'(quantitative, nominal), 'steps_per_minute'(quantitative, numerical), 'carb_prop'(quantitative, numerical), 'protein_prop'(quantitative, numerical).
We standardize the numerical features using StandardScaler.

In [ ]:
final_pipeline = Pipeline([
    ('preprocessor', preprocessor_final),
    ('model', RandomForestRegressor(random_state=42, n_jobs=-1))
])

param_dist = {
    'model__n_estimators': [50, 100, 200],
    'model__max_depth': [5, 10, 15],
    'model__min_samples_split': [2, 5, 10],
    'model__min_samples_leaf': [1, 2, 4]
}

random_search_final = RandomizedSearchCV(
    final_pipeline, param_distributions=param_dist, n_iter=10,
    cv=3, scoring='neg_mean_absolute_error', n_jobs=-1, verbose=2
)

print("\n Running Final Model Hyperparameter Search...")
random_search_final.fit(X_train_final, y_train)

y_pred_final = random_search_final.best_estimator_.predict(X_test_final)
mae_final = mean_absolute_error(y_test, y_pred_final)
r2_final = r2_score(y_test, y_pred_final)

print("\nBest Hyperparameters:", random_search_final.best_params_)
print(f"Final Model MAE: {mae_final:.4f}")
print(f"Final Model R²: {r2_final:.4f}")

We see that MAE reduced and R² increased. This means that our final model improved. Below, we can see what the most important features were.

In [ ]:
best_model = random_search_final.best_estimator_
feature_importance = best_model.named_steps['model'].feature_importances_
final_feature_names = best_model[:-1].get_feature_names_out()

print("Extracted Features:", final_feature_names)
print("Feature Importance Length:", len(feature_importance))

clean_feature_names = [name.replace("num__", "").replace("remainder__", "") for name in final_feature_names]


importance_df = pd.DataFrame({
    'Feature': clean_feature_names,  
    'Importance': feature_importance
}).sort_values(by='Importance', ascending=False)

fig = px.bar(
    importance_df, 
    x='Importance', 
    y='Feature', 
    text=importance_df['Importance'].round(4), 
    title="Feature Importance in Final Model"
)
fig.update_traces(marker_color='royalblue', textposition='outside')
fig.show()

## Step 8: Fairness Analysis

The goal of this analysis is to determine whether our model exhibits bias in predicting recipe ratings based on the carb proportion (carb_prop) in a recipe. We split recipes into two groups:

High-carb recipes: Those with a carb_prop greater than the mean.
Low-carb recipes: Those with a carb_prop less than or equal to the mean.
We evaluate whether the model's performance (Mean Absolute Error, MAE) differs significantly between these two groups.


**Null Hypothesis:** The model is fair. Its MAE for high-carb and low-carb recipes is roughly the same, and any differences are due to random chance.

**Alternative Hypothesis:** The model is unfair. It performs significantly better (lower MAE) for low-carb recipes compared to high-carb recipes.

**Significance level:** 0.05

In [ ]:
mean_carb = X_test['carb_prop'].mean()
is_high_carb = X_test['carb_prop'] > mean_carb

y_test_high = y_test[is_high_carb]
y_pred_high = y_pred_final[is_high_carb]
mae_high = mean_absolute_error(y_test_high, y_pred_high)

y_test_low = y_test[~is_high_carb]
y_pred_low = y_pred_final[~is_high_carb]
mae_low = mean_absolute_error(y_test_low, y_pred_low)

observed_diff = mae_low - mae_high
print(f"Mean carb_prop: {mean_carb:.4f}")
print(f"MAE for high carb_prop recipes: {mae_high:.4f}")
print(f"MAE for low carb_prop recipes: {mae_low:.4f}")
print(f"Observed difference in MAE (low - high): {observed_diff:.4f}")

n_permutations = 1000
diff_permutations = []
is_high_carb_array = is_high_carb.values.copy()
for _ in range(n_permutations):
    np.random.shuffle(is_high_carb_array)
    
    mae_high_perm = mean_absolute_error(y_test[is_high_carb_array], y_pred_final[is_high_carb_array])
    mae_low_perm = mean_absolute_error(y_test[~is_high_carb_array], y_pred_final[~is_high_carb_array])
    
    diff_permutations.append(mae_low_perm - mae_high_perm)


p_value = np.mean([diff <= observed_diff for diff in diff_permutations])  # Test if low carb has lower MAE
print(f"p-value: {p_value:.4f}")

plt.figure(figsize=(10, 6))
plt.hist(diff_permutations, bins=30, alpha=0.7, color='blue')
plt.axvline(observed_diff, color='red', linestyle='--', label='Observed statistic')
plt.title('Empirical Distribution of the Difference in MAE (Low Carb - High Carb)')
plt.xlabel('Difference in MAE')  
plt.ylabel('Frequency')  
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

alpha = 0.05
if p_value < alpha:
    print("The model performs significantly better for recipes with low carb_prop.")
else:
    print("No significant evidence that the model performs better for low carb recipes.")